# Data wrangling for "world_bank_project" Json file

data source: http://jsonstudio.com/resources/

# Import for Pyhton, Pandas and Json

In [1]:
#import pandas and json
import pandas as pd
import json

# Reading Json file

In [2]:
#loading json file and printing the first two rows
df = pd.read_json('world_bank_projects.json')

# Data Wrangling

# Find the 10 countries with most projects

In [3]:
# counting the number of projects for each country
country_count = df['countryname'].value_counts()

# 10 countries with most projects
country_count.head(10)

Republic of Indonesia              19
People's Republic of China         19
Socialist Republic of Vietnam      17
Republic of India                  16
Republic of Yemen                  13
Nepal                              12
Kingdom of Morocco                 12
People's Republic of Bangladesh    12
Republic of Mozambique             11
Africa                             11
Name: countryname, dtype: int64

# Create a dataframe with the missing names filled in

In [4]:
# finding the data structure of project theme variable 'mjtheme_namecode' in the file
project_themes = df[['mjtheme_namecode']] 
project_themes.head(5)

,mjtheme_namecode
0,"[{'code': '8', 'name': 'Human development'}, {..."
1,"[{'code': '1', 'name': 'Economic management'},..."
2,"[{'code': '5', 'name': 'Trade and integration'..."
3,"[{'code': '7', 'name': 'Social dev/gender/incl..."
4,"[{'code': '5', 'name': 'Trade and integration'..."


In [5]:
# loops to parse project_themes into nested lists
for index, row in project_themes.iterrows():
    L = list()
    for i in range(index+1):
        a = [list(dict.values()) for dict in list(project_themes.loc[i].items())[0][1] ] #returning dict value into list
        L.append(a)

In [6]:
L[0]     #checking the data structure of L: [[[], []...]]

[['8', 'Human development'], ['11', '']]

In [7]:
from itertools import chain
lists = list(chain.from_iterable(L)) #exacting nested lists to be "[[], []...]"
df1 = pd.DataFrame(lists, columns=['code', 'name']) # changing the lists to Pandas dataframe

In [8]:
df1.head()

,code,name
0,8,Human development
1,11,
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


In [9]:
#filling blank with NaN value
import numpy as np
df1 = df1.replace('', np.nan, regex=True)
df1.head()

,code,name
0,8,Human development
1,11,NaN
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


In [10]:
df1 = df1.fillna(df1['name'].value_counts().index[0])   #filling NaN valure with the most frequent values in the group
df1.head()   #checing if NaN valures have been filled 

,code,name
0,8,Human development
1,11,Environment and natural resources management
2,1,Economic management
3,6,Social protection and risk management
4,5,Trade and integration


# Find the top 10 major project themes (using column 'mjtheme_namecode')

In [11]:
df2 = df1.groupby('name', as_index=False).count()
df2 = df2.sort_values('code', ascending=False)

In [12]:
df3 = df2.rename(columns={'code':'count'})
df3.head(10)

,name,count
1,Environment and natural resources management,345
6,Rural development,202
3,Human development,197
4,Public sector governance,184
8,Social protection and risk management,158
2,Financial and private sector development,130
7,Social dev/gender/inclusion,119
9,Trade and integration,72
10,Urban development,47
0,Economic management,33
